In [1]:
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
def get_yolo3_model(model_type, num_feature_layers, num_anchors, num_classes,
                   input_tensor=None, input_shape=None):
    # prepare input tensor
    if input_shape:
        input_tensor = Input(shape=input_shape, name='image_input')
        
    if input_tensor is None:
        input_tensor = Input(shape=(None, None, 3), name='image_input')
        
    model_function = yolo4_body
    backbone_len = 250
    weights_path = 'weights/cspdarknet53.h5'
    
    if weights_path:
        model_body = yolo4_body(input_tensor, num_anchors//3, num_classes, weights_path=weights_path)
    
    

In [ ]:
def get_yolo3_inference_model(model_type, anchors, num_classes, weights_path=None,
                              input_shape=None, confidence=0.1, iou_threshold=0.4, elim_grid_sense=False):
    """create the inference model, for YOLOv3"""
    num_anchors = len(anchors)
    num_feature_layers = num_anchors //3
    
    image_shape = Input(shape=(2,), dtype='int64', name='image_shape')
    # TODO: get_yolo3_mode
    model_body, _ = get_yolo3_model(model_type, num_feature_layers, num_anchors, num_classes, input_shape=input_shape)
    
    if weights_path:
        model_body.load_weights(weights_path, by_name=False)
        print('Load weight {}'.format(weights_path))
        
    # TODO: batch_yolo_postprocess
    boxes, scores, classes = Lambda(batch_yolo_postprocess, name='yolo3_postprocess',
                                    arguments={
                                        'anchors':anchors, 
                                        'num_classes':num_classes, 
                                        'confidence':confidence,
                                        'iou_threshold': iou_threshold,
                                        'elim_grid_sense':elim_grid_sense
                                    })([*model_body.output, image_shape])
    model = Model([model_body.input, image_shape], [boxes, scores, classes])
    
    return model
    
                                        